In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import time
import os

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# base_url = 'https://en.zalando.de'
def get_url(url=''):
    if base_url not in url:
        url = f"{base_url}{url}"
    return BeautifulSoup(requests.get(
        url,
        verify=False,
        timeout=5,
        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'en-US,en;q=0.9',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        }
    ).content, 'html.parser')

# CnA

In [134]:
base_url = 'https://www.c-and-a.com/de/de/shop/'
filename_format = '../data/CnA_{}.parquet'
img_filename = '../images/CnA_{}{}'

### Getting categories for Women

In [5]:
top_cat = 'Women'

In [6]:
main = get_url('damen-bekleidung')

In [7]:
cur_idx = 0

In [8]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

if len(main.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
    holder = main.find_all('section', attrs={'class': 'product-overview__head-filter'})
    for li in holder[0].find_all('button'):
        li_name = li.text.strip()
        li_name = li_name[:li_name.find('\n')]
#         li_name = re.search('([\w ]*)', li.text.strip())[0]
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('data-redirect'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1
else:
    holder = main.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
    for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
        li_name = li.get('title')
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('href'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1


Getting sub-categories

In [9]:
# lists to store all filter names and values
# filters are extracted at parent category level
filter_names = []
filter_options = []
filter_parents = []

get_filter_options = lambda x: f"{x.get('data-facet')}:{x.get('data-value')}"

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    if len(cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
        holder = cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})
        for li in holder[0].find_all('button'):
            li_name = li.text.strip()
            li_name = li_name[:li_name.find('\n')]
    #         li_name = re.search('([\w ]*)', li.text.strip())[0]
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('data-redirect'))
            parents.append(c)
            cur_idx += 1
    else:
        holder = cat_html.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
        for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
            li_name = li.get('title')
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('href'))
            parents.append(c)
            cur_idx += 1
        
        
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class': 'product-overview__facets overlay__content', 'data-widget-id': '2-2'})[0]

    for f in filters_.find_all('div', attrs={'class': 'toggle__content is-hidden'}):
        options = list(map(get_filter_options, f.find_all(attrs={'data-facet': True, 'data-value': True})))
        filter_options.extend(options)
        filter_names.extend([f.find('span').text]*len(options))
        filter_parents.extend([c]*len(options))


Creating dataframe of categories and sub-categories

In [10]:
if os.path.exists(filename_format.format('Categories')):
    cats = pd.read_parquet(filename_format.format('Categories'))
else:
    cats = pd.DataFrame(columns=['Category', 'URL', 'Parent_Category', 'Top_Category', 'Skip'])
    
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})])

In [11]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Alle Produkte,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
1,NEU in Bekleidung,https://www.c-and-a.com/de/de/shop/damen-neu-i...,NaN,Women,False
2,Kleider & Jumpsuits,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
3,Blusen,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
4,T-Shirts & Tops,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False


In [12]:
cats.shape

(308, 5)

### Getting categories for Men

In [13]:
top_cat = 'Men'

In [14]:
main = get_url('herren-bekleidung')

In [15]:
cur_idx = cats.index[-1]+1

In [16]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

if len(main.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
    holder = main.find_all('section', attrs={'class': 'product-overview__head-filter'})
    for li in holder[0].find_all('button'):
        li_name = li.text.strip()
        li_name = li_name[:li_name.find('\n')]
#         li_name = re.search('([\w ]*)', li.text.strip())[0]
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('data-redirect'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1
else:
    holder = main.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
    for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
        li_name = li.get('title')
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('href'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1


Getting sub-categories

In [17]:
# lists to store all filter names and values
# filters are extracted at parent category level
get_filter_options = lambda x: f"{x.get('data-facet')}:{x.get('data-value')}"

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    if len(cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
        holder = cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})
        for li in holder[0].find_all('button'):
            li_name = li.text.strip()
            li_name = li_name[:li_name.find('\n')]
    #         li_name = re.search('([\w ]*)', li.text.strip())[0]
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('data-redirect'))
            parents.append(c)
            cur_idx += 1
    else:
        holder = cat_html.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
        for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
            li_name = li.get('title')
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('href'))
            parents.append(c)
            cur_idx += 1
    
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class': 'product-overview__facets overlay__content', 'data-widget-id': '2-2'})[0]

    for f in filters_.find_all('div', attrs={'class': 'toggle__content is-hidden'}):
        options = list(map(get_filter_options, f.find_all(attrs={'data-facet': True, 'data-value': True})))
        filter_options.extend(options)
        filter_names.extend([f.find('span').text]*len(options))
        filter_parents.extend([c]*len(options))


Updating dataframe of categories and sub-categories

In [18]:
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})]).reset_index().drop(columns='index')

In [19]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Alle Produkte,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
1,NEU in Bekleidung,https://www.c-and-a.com/de/de/shop/damen-neu-i...,NaN,Women,False
2,Kleider & Jumpsuits,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
3,Blusen,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
4,T-Shirts & Tops,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False


In [20]:
cats.shape

(492, 5)

### Getting categories for Baby

In [21]:
top_cat = 'Baby'

In [22]:
main = get_url('babys-bekleidung')

In [23]:
cur_idx = cats.index[-1]+1

In [24]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

if len(main.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
    holder = main.find_all('section', attrs={'class': 'product-overview__head-filter'})
    for li in holder[0].find_all('button'):
        li_name = li.text.strip()
        li_name = li_name[:li_name.find('\n')]
#         li_name = re.search('([\w ]*)', li.text.strip())[0]
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('data-redirect'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1
else:
    holder = main.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
    for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
        li_name = li.get('title')
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('href'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1


Getting sub-categories

In [25]:
# lists to store all filter names and values
# filters are extracted at parent category level
get_filter_options = lambda x: f"{x.get('data-facet')}:{x.get('data-value')}"

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    if len(cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
        holder = cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})
        for li in holder[0].find_all('button'):
            li_name = li.text.strip()
            li_name = li_name[:li_name.find('\n')]
    #         li_name = re.search('([\w ]*)', li.text.strip())[0]
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('data-redirect'))
            parents.append(c)
            cur_idx += 1
    else:
        holder = cat_html.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
        for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
            li_name = li.get('title')
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('href'))
            parents.append(c)
            cur_idx += 1
    
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class': 'product-overview__facets overlay__content', 'data-widget-id': '2-2'})[0]

    for f in filters_.find_all('div', attrs={'class': 'toggle__content is-hidden'}):
        options = list(map(get_filter_options, f.find_all(attrs={'data-facet': True, 'data-value': True})))
        filter_options.extend(options)
        filter_names.extend([f.find('span').text]*len(options))
        filter_parents.extend([c]*len(options))


Updating dataframe of categories and sub-categories

In [26]:
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})]).reset_index().drop(columns='index')

In [27]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Alle Produkte,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
1,NEU in Bekleidung,https://www.c-and-a.com/de/de/shop/damen-neu-i...,NaN,Women,False
2,Kleider & Jumpsuits,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
3,Blusen,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
4,T-Shirts & Tops,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False


In [28]:
cats.shape

(677, 5)

### Getting categories for Kids - Girls

In [29]:
top_cat = 'Kids-Girls'

In [30]:
main = get_url('kinder-maedchen-bekleidung')

In [31]:
cur_idx = cats.index[-1]+1

In [32]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

if len(main.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
    holder = main.find_all('section', attrs={'class': 'product-overview__head-filter'})
    for li in holder[0].find_all('button'):
        li_name = li.text.strip()
        li_name = li_name[:li_name.find('\n')]
#         li_name = re.search('([\w ]*)', li.text.strip())[0]
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('data-redirect'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1
else:
    holder = main.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
    for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
        li_name = li.get('title')
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('href'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1


Getting sub-categories

In [33]:
# lists to store all filter names and values
# filters are extracted at parent category level
get_filter_options = lambda x: f"{x.get('data-facet')}:{x.get('data-value')}"

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    if len(cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
        holder = cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})
        for li in holder[0].find_all('button'):
            li_name = li.text.strip()
            li_name = li_name[:li_name.find('\n')]
    #         li_name = re.search('([\w ]*)', li.text.strip())[0]
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('data-redirect'))
            parents.append(c)
            cur_idx += 1
    else:
        holder = cat_html.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
        for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
            li_name = li.get('title')
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('href'))
            parents.append(c)
            cur_idx += 1
    
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class': 'product-overview__facets overlay__content', 'data-widget-id': '2-2'})[0]

    for f in filters_.find_all('div', attrs={'class': 'toggle__content is-hidden'}):
        options = list(map(get_filter_options, f.find_all(attrs={'data-facet': True, 'data-value': True})))
        filter_options.extend(options)
        filter_names.extend([f.find('span').text]*len(options))
        filter_parents.extend([c]*len(options))


Updating dataframe of categories and sub-categories

In [34]:
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})]).reset_index().drop(columns='index')

In [35]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Alle Produkte,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
1,NEU in Bekleidung,https://www.c-and-a.com/de/de/shop/damen-neu-i...,NaN,Women,False
2,Kleider & Jumpsuits,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
3,Blusen,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
4,T-Shirts & Tops,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False


In [36]:
cats.shape

(869, 5)

### Getting categories for Kids - Boys

In [37]:
top_cat = 'Kids-Boys'

In [38]:
main = get_url('kinder-jungen-bekleidung')

In [39]:
cur_idx = cats.index[-1]+1

In [40]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

if len(main.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
    holder = main.find_all('section', attrs={'class': 'product-overview__head-filter'})
    for li in holder[0].find_all('button'):
        li_name = li.text.strip()
        li_name = li_name[:li_name.find('\n')]
#         li_name = re.search('([\w ]*)', li.text.strip())[0]
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('data-redirect'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1
else:
    holder = main.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
    for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
        li_name = li.get('title')
    #     li_name = li.a.get_text()[4:].strip()
        names.append(li_name)
        urls.append(li.get('href'))
        parents.append(None)
        main_cats.append(cur_idx)
        cur_idx += 1


Getting sub-categories

In [41]:
# lists to store all filter names and values
# filters are extracted at parent category level
get_filter_options = lambda x: f"{x.get('data-facet')}:{x.get('data-value')}"

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    if len(cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})) > 0:
        holder = cat_html.find_all('section', attrs={'class': 'product-overview__head-filter'})
        for li in holder[0].find_all('button'):
            li_name = li.text.strip()
            li_name = li_name[:li_name.find('\n')]
    #         li_name = re.search('([\w ]*)', li.text.strip())[0]
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('data-redirect'))
            parents.append(c)
            cur_idx += 1
    else:
        holder = cat_html.find_all('div', attrs={'class': 'product-overview__categories-wrapper'})
        for li in holder[0].find('div', attrs={'class':'toggle__content'}).find_all('a', attrs={'class': False}):
            li_name = li.get('title')
        #     li_name = li.a.get_text()[4:].strip()
            names.append(li_name)
            urls.append(li.get('href'))
            parents.append(c)
            cur_idx += 1
    
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class': 'product-overview__facets overlay__content', 'data-widget-id': '2-2'})[0]

    for f in filters_.find_all('div', attrs={'class': 'toggle__content is-hidden'}):
        options = list(map(get_filter_options, f.find_all(attrs={'data-facet': True, 'data-value': True})))
        filter_options.extend(options)
        filter_names.extend([f.find('span').text]*len(options))
        filter_parents.extend([c]*len(options))


Updating dataframe of categories and sub-categories

In [42]:
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})]).reset_index().drop(columns='index')

In [43]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Alle Produkte,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
1,NEU in Bekleidung,https://www.c-and-a.com/de/de/shop/damen-neu-i...,NaN,Women,False
2,Kleider & Jumpsuits,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
3,Blusen,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
4,T-Shirts & Tops,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False


In [44]:
cats.shape

(1053, 5)

### Removing categories which we dont want

In [45]:
cats.groupby('Top_Category').count()

,Category,URL,Parent_Category,Skip
Top_Category,,,,
Baby,185,185,169,185
Kids-Boys,184,184,166,184
Kids-Girls,192,192,173,192
Men,184,184,166,184
Women,308,308,284,308


In [46]:
cats[cats['Parent_Category'].isna()]['Category'].unique()

array(['Alle Produkte', 'NEU in Bekleidung', 'Kleider & Jumpsuits',
       'Blusen', 'T-Shirts & Tops', 'Pullover & Strickjacken', 'Jeans',
       'Shorts', 'Hosen', 'Röcke', 'Sportbekleidung', 'Loungewear',
       'Jacken', 'Blazer', 'Bademode', 'Unterwäsche', 'Nachtwäsche',
       'Socken & Strumpfhosen', 'Premium', 'Basics', 'Business-Mode',
       'Abendmode', 'Kaschmir', 'Home', 'T-Shirts & Polos', 'Hemden',
       'Sweatshirts & Sweatjacken', 'Anzüge', 'Pyjamas', 'Socken',
       'NEU in Baby Bekleidung', 'Bodies', 'Outfits', 'Oberteile',
       'Unterteile', 'Kleider & Röcke', 'Multipacks', 'Comics & Marken',
       'Accessoires', 'Schuhe', 'Schneeanzüge',
       'NEU in Mädchen Bekleidung', 'T-Shirts & Blusen', 'Wäsche', 'Sets',
       'Pullover & Sweatshirts', 'Skibekleidung', 'Sportliche Styles',
       'NEU in Jungen Bekleidung', 'T-Shirts & Hemden'], dtype=object)

In [47]:
cats_to_delete = [
    'Alle Produkte', 'NEU in Bekleidung', 'Premium', 'Home', 'NEU in Baby Bekleidung', 
    'Bodies', 'Comics & Marken', 'NEU in Mädchen Bekleidung', 'Sportliche Styles', 'NEU in Jungen Bekleidung'
]

In [48]:
idx_delete = cats[(cats['Category'].isin(cats_to_delete))].index
idx_delete

Int64Index([   0,    1,   18,   23,   24,   25,   42,   47,   48,   49,
            ...
            1010, 1014, 1017, 1018, 1033, 1034, 1035, 1036, 1051, 1052],
           dtype='int64', length=199)

In [49]:
# delete parent categories
cats.drop(labels=idx_delete, axis=0, inplace=True)

#delete the children
cats.drop(labels = cats[cats['Parent_Category'].isin(idx_delete)].index, axis=0, inplace=True)

In [50]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
2,Kleider & Jumpsuits,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
3,Blusen,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
4,T-Shirts & Tops,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
5,Pullover & Strickjacken,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False
6,Jeans,https://www.c-and-a.com/de/de/shop/damen-bekle...,NaN,Women,False


In [51]:
cats.shape

(578, 5)

In [52]:
# Check count of parent categories and child categories per top category\
cats.groupby('Top_Category').count()

,Category,URL,Parent_Category,Skip
Top_Category,,,,
Baby,106,106,94,106
Kids-Boys,91,91,77,91
Kids-Girls,94,94,79,94
Men,108,108,93,108
Women,179,179,159,179


In [53]:
cats[(cats['Top_Category'] == 'Kids-Girls') & (~cats['Parent_Category'].isna())].head()

,Category,URL,Parent_Category,Top_Category,Skip
735,Oberteile,https://www.c-and-a.com/de/de/shop/kinder-maed...,679.0,Kids-Girls,False
736,Unterteile,https://www.c-and-a.com/de/de/shop/kinder-maed...,679.0,Kids-Girls,False
737,Kleider & Röcke,https://www.c-and-a.com/de/de/shop/kinder-maed...,679.0,Kids-Girls,False
739,Multipacks,https://www.c-and-a.com/de/de/shop/kinder-maed...,680.0,Kids-Girls,False
740,T-Shirts,https://www.c-and-a.com/de/de/shop/kinder-maed...,680.0,Kids-Girls,False


In [54]:
# saving categories
cats.to_parquet(filename_format.format('Categories'))

Creating dataframe of fitler options per category

In [55]:
filters = pd.DataFrame({
    'Parent_Category': filter_parents,
    'Filter_Name': filter_names,
    'Filter_Option': filter_options,
})

In [56]:
filters.shape

(9181, 3)

In [57]:
filters_t = filters[filters['Parent_Category'].isin(cats[cats['Parent_Category'].isna()].index)]

In [58]:
filters_t.head(10)

,Parent_Category,Filter_Name,Filter_Option
523,2,SORTIEREN,sort:3
524,2,SORTIEREN,sort:4
525,2,SORTIEREN,sort:5
526,2,SORTIEREN,sort:6
527,2,SORTIEREN,sort:7
528,2,Größe,ads_f10039_ntk_cs:XS
529,2,Größe,ads_f10039_ntk_cs:S
530,2,Größe,ads_f10039_ntk_cs:M
531,2,Größe,ads_f10039_ntk_cs:L
532,2,Größe,ads_f10039_ntk_cs:XL


In [59]:
filters_t.shape

(6556, 3)

In [60]:
filters_t['Filter_Name'].unique()

array(['SORTIEREN', 'Größe', 'Farbe', 'Preis', 'Nachhaltigkeit',
       'Material', 'Sale', 'Multipacks', 'Kollektion', 'Kragen',
       'Ausschnitt', 'Ärmellänge', 'Länge', 'Passform', 'Muster', 'Marke',
       'Anlass', 'Beinform', 'Bundhöhe', 'Besonderheiten', 'Absatzhöhe',
       'Produktlinie', 'BH-Form', 'Comics & Cartoons', 'Manschette',
       'Kinder - Geschlecht', 'Altersgruppe', 'Geschlecht', 'Für wen?'],
      dtype=object)

In [61]:
filters_to_keep = [
    'Farbe', 'Multipacks', 'Kragen', 'Ausschnitt', 'Ärmellänge', 'Länge', 'Passform', 
    'Muster',  'Anlass', 'Beinform', 'Bundhöhe', 'Absatzhöhe', 'BH-Form', 'Comics & Cartoons', 
    'Manschette', 'Kinder - Geschlecht', 'Altersgruppe', 'Geschlecht', 'Für wen?'
]

In [62]:
filters_t = filters_t[filters_t['Filter_Name'].isin(filters_to_keep)]

In [63]:
filters_t.shape

(3203, 3)

In [64]:
filters_t.head()

,Parent_Category,Filter_Name,Filter_Option
544,2,Farbe,ads_f10037_ntk_cs:Wei%C3%9F
545,2,Farbe,ads_f10037_ntk_cs:Schwarz
546,2,Farbe,ads_f10037_ntk_cs:Rot
547,2,Farbe,ads_f10037_ntk_cs:Blau
548,2,Farbe,ads_f10037_ntk_cs:Gelb


In [65]:
filters_t.groupby('Parent_Category').count()

,Filter_Name,Filter_Option
Parent_Category,,
2,99,99
3,68,68
4,63,63
5,55,55
6,55,55
...,...,...
880,27,27
881,29,29
882,36,36


In [66]:
# saving filters
filters_t.to_parquet(filename_format.format('Filters'))

In [74]:
filters_t[filters_t['Filter_Name'] == 'Farbe']['Filter_Option'].unique()

array(['ads_f10037_ntk_cs:Wei%C3%9F', 'ads_f10037_ntk_cs:Schwarz',
       'ads_f10037_ntk_cs:Rot', 'ads_f10037_ntk_cs:Blau',
       'ads_f10037_ntk_cs:Gelb', 'ads_f10037_ntk_cs:Gr%C3%BCn',
       'ads_f10037_ntk_cs:Lila', 'ads_f10037_ntk_cs:Pink',
       'ads_f10037_ntk_cs:Rosa', 'ads_f10037_ntk_cs:Orange',
       'ads_f10037_ntk_cs:T%C3%BCrkis', 'ads_f10037_ntk_cs:Grau',
       'ads_f10037_ntk_cs:Braun', 'ads_f10037_ntk_cs:Beige',
       'ads_f10037_ntk_cs:Multicolor', 'ads_f10037_ntk_cs:Metallisch',
       'ads_f10037_ntk_cs:Kariert'], dtype=object)

Iterate through all sub-categories - get all products (images & ids)

In [138]:
# lamda functions for getting various params
# img_url = lambda x: x.find_all('img')[0].get('src') if x.find_all('img')[0].get('src') else x.find_all('img')[0].get('data-src')
# img_ext = lambda x: re.search('set=source\[[a-z0-9/]*(\.[a-z]*)]', img_url(x))[1]
# item_name = lambda x: x.find_all('a')[0].get('title')
# item_url = lambda x: x.find_all('a')[0].get('href')
# item_code = lambda x: x.get('data-articlecode')

def save_image(code, img, ext, verbose=False):
    target = img_filename.format(code, ext)
#     target = f"../images/{code}{ext}"
    if os.path.exists(target):
#         print("Already there")
        return 0
    if verbose:
        print(f"Img not found - {code}.", end=' ')
    img_data = requests.get(img if img[:6] == 'https:' else f"https:{img}")
    if img_data.status_code != 200:
        return 0
    with open(target, 'wb') as f:
        f.write(img_data.content)
        if verbose:
            print(f"Downloaded.")
        return 1

In [132]:
skip = []
cats.loc[skip, 'Skip'] = True

In [ ]:
# popImageType=still&facets=ads_f10039_ntk_cs:3XL;ads_f20501_ntk_cs:2er+Pack;&pagesize=2000

In [147]:
size = 2000

if os.path.exists(filename_format.format('Items')):
    df_items = pd.read_parquet(filename_format.format('Items'))
else:
    df_items = pd.DataFrame(columns=['Code', 'Name', 'URL', 'Img_URL', 'Img_Ext', 'Category'])

i = 0
total = cats[~cats['Parent_Category'].isna()].shape[0]

print(f'Total Categories = {total}')
               
for idx, row in cats[(~cats['Parent_Category'].isna()) & (cats['Skip'] == False)].iterrows():
    if df_items[df_items['Category'] == idx].shape[0] > 0:
        # already completed,
        i += 1
        print(f"Skipping {idx} - {row['Top_Category']} - {row['Category']}")
        continue
    
    subcat = get_url(f"{row['URL']}?popImageType=still&pagesize={size}")
    all_items = subcat.find_all('div', attrs={'class':'product-tile'})
#     print(f"Found {len(all_items)} items.")

    codes = []
    urls = []
    img_urls = []
    img_exts = []
    item_names = []
    for item in all_items:
        # find different colors
        item_code = item.get('data-usim')
        colors = item.find('ul', attrs={'class':"color-list"}).find_all(
            'li', attrs={'data-url': True, 'data-color': True, 'data-normal-source-xl-src': True}
        )
        
        item_codes = list(map(lambda x: f"{item_code}_{x.get('data-color')}", colors))
        item_urls = list(map(lambda x: f"{x.get('data-url')}", colors))
        item_img_urls = list(map(lambda x: f"{x.get('data-normal-source-xl-src')}", colors))
        
        if len(item_img_urls) == 0 or item_img_urls[0] is None or len(item_img_urls) == 0:
            # skip this one
            continue
        
        assert(len(item_codes) == len(item_img_urls), f"Failed to get proper mappings for {row['URL']}")
        assert(len(item_codes) == len(item_urls), f"Failed to get proper mappings for {row['URL']}")
        
        codes.extend(item_codes)
        urls.extend(item_urls)
        img_urls.extend(item_img_urls)
        img_exts.extend(list(map(lambda x: x[-4:], item_img_urls)))

        item_name = item.find('p', attrs={'class': 'product-tile__title'}).text
        item_names.extend([item_name]*len(item_codes))
    
    img_saved = sum(list(map(save_image, codes, img_urls, img_exts)))
    
    df_items = pd.concat([df_items, pd.DataFrame({
        'Code': codes,
        'Name': item_names,
        'URL': urls,
        'Img_URL': img_urls,
        'Img_Ext': img_exts,
        'Category': [idx]*len(codes)
    })], ignore_index=True)
    
    i += 1
    print(f"Completed {idx} - {row['Category']} - Saved {len(codes)} items. Images saved = {img_saved}.")
    if i%10 == 0:
        df_items.to_parquet(filename_format.format('Items'))
        time.sleep(1)

df_items.to_parquet(filename_format.format('Items'))
print("All Done!")

Total Categories = 502
Skipping 73 - Women - Freizeitkleider
Skipping 74 - Women - Strickkleider
Skipping 75 - Women - Kurze Kleider
Skipping 76 - Women - Midikleider
Skipping 77 - Women - Maxikleider
Skipping 78 - Women - Jumpsuits
Skipping 79 - Women - Hochzeitskleider
Skipping 80 - Women - Abendkleider
Skipping 81 - Women - A-Linien Kleider
Completed 82 - Das perfekte Klei - Saved 0 items. Images saved = 0.
Skipping 84 - Women - Blusen
Skipping 85 - Women - Blusentops
Skipping 86 - Women - Hemdblusen
Skipping 88 - Women - Tops
Skipping 89 - Women - T-Shirts
Skipping 90 - Women - Langarmshirts
Skipping 91 - Women - Multipacks
Skipping 93 - Women - Pullover
Skipping 94 - Women - Strickjacken
Skipping 95 - Women - Ponchos
Skipping 96 - Women - Sweatshirts
Skipping 97 - Women - Sweatjacken
Skipping 99 - Women - Skinny Jeans
Skipping 100 - Women - Slim Jeans
Skipping 101 - Women - Straight Jeans
Skipping 102 - Women - Bootcut Jeans
Skipping 103 - Women - Jeggings
Skipping 104 - Women - J

Completed 447 - Hosen - Saved 277 items. Images saved = 0.
Completed 448 - Jeans - Saved 157 items. Images saved = 0.
Completed 449 - Shorts - Saved 102 items. Images saved = 0.
Completed 450 - Anzüge - Saved 116 items. Images saved = 0.
Completed 451 - Jacken - Saved 221 items. Images saved = 0.
Completed 452 - Sportbekleidung - Saved 145 items. Images saved = 0.
Completed 453 - Bademode - Saved 14 items. Images saved = 0.
Completed 454 - Pyjamas - Saved 101 items. Images saved = 0.
Completed 455 - Unterwäsche - Saved 132 items. Images saved = 0.
Completed 456 - Socken - Saved 104 items. Images saved = 0.
Completed 457 - Basics - Saved 250 items. Images saved = 0.
Completed 460 - Trunks - Saved 65 items. Images saved = 0.
Completed 461 - Boxershorts - Saved 28 items. Images saved = 0.
Completed 462 - Slips - Saved 14 items. Images saved = 0.
Completed 463 - Unterhemden - Saved 22 items. Images saved = 0.
Completed 464 - Skiunterwäsche - Saved 3 items. Images saved = 0.
Completed 465 -

Completed 748 - Röcke - Saved 23 items. Images saved = 15.
Completed 750 - Jogginghosen - Saved 17 items. Images saved = 12.
Completed 751 - Stoffhosen - Saved 28 items. Images saved = 18.
Completed 752 - Leggings - Saved 64 items. Images saved = 20.
Completed 753 - Multipacks - Saved 51 items. Images saved = 1.
Completed 754 - Thermohosen & -jeans - Saved 6 items. Images saved = 2.
Completed 755 - Shorts - Saved 36 items. Images saved = 21.
Completed 757 - Skinny Jeans - Saved 15 items. Images saved = 12.
Completed 758 - Jeggings - Saved 9 items. Images saved = 4.
Completed 759 - Straight Jeans - Saved 4 items. Images saved = 4.
Completed 760 - Relaxed Jeans - Saved 2 items. Images saved = 2.
Completed 761 - Jeans-Shorts - Saved 15 items. Images saved = 0.
Completed 763 - Jogginghosen - Saved 17 items. Images saved = 0.
Completed 764 - Stoffhosen - Saved 28 items. Images saved = 0.
Completed 765 - Leggings - Saved 64 items. Images saved = 0.
Completed 766 - Multipacks - Saved 51 items

Completed 997 - Sakkos & Blazer - Saved 9 items. Images saved = 0.
Completed 998 - Winterjacken - Saved 19 items. Images saved = 0.
Completed 1000 - Hüte & Mützen - Saved 21 items. Images saved = 0.
Completed 1001 - Handschuhe & Sets - Saved 4 items. Images saved = 0.
Completed 1002 - Schals & Tücher - Saved 5 items. Images saved = 0.
Completed 1003 - Taschen & Rucksäcke - Saved 3 items. Images saved = 0.
Completed 1004 - Uhren - Saved 3 items. Images saved = 0.
Completed 1005 - Gürtel - Saved 3 items. Images saved = 0.
Completed 1006 - Sonnenbrillen - Saved 3 items. Images saved = 0.
Completed 1007 - Krawatten & Fliegen - Saved 1 items. Images saved = 0.
Completed 1008 - Kuscheltiere - Saved 4 items. Images saved = 0.
Completed 1009 - Schuhe & Sonstiges - Saved 9 items. Images saved = 0.
Completed 1011 - Skijacken - Saved 3 items. Images saved = 0.
Completed 1012 - Ski Accessoires - Saved 11 items. Images saved = 0.
Completed 1013 - Skiunterwäsche - Saved 5 items. Images saved = 0.
Co

In [148]:
df_items

,Code,Name,URL,Img_URL,Img_Ext,Category
0,2131540_1,Fit & Flare Kleid - Bio-Baumwolle - bestickt,https://www.c-and-a.com/de/de/shop/fit-flare-k...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,73.0
1,2101813_1,A-Linien Kleid - bestickt,https://www.c-and-a.com/de/de/shop/a-linien-kl...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,73.0
2,2165721_2,Fit & Flare Kleid - LENZING™ ECOVERO™ - geblümt,https://www.c-and-a.com/de/de/shop/fit-flare-k...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,73.0
3,2165721_1,Fit & Flare Kleid - LENZING™ ECOVERO™ - geblümt,https://www.c-and-a.com/de/de/shop/fit-flare-k...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,73.0
4,2145750_1,Fit & Flare Kleid - Bio-Baumwolle - geblümt,https://www.c-and-a.com/de/de/shop/fit-flare-k...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,73.0
...,...,...,...,...,...,...
41718,2109118_2,Super Skinny Jeans,https://www.c-and-a.com/de/de/shop/the-super-s...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,1016.0
41719,2063786_1,Multipack 2er - Jogginghose,https://www.c-and-a.com/de/de/shop/jogginghose...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,1016.0
41720,2010490_1,Skinny Jeans - Bio-Baumwolle,https://www.c-and-a.com/de/de/shop/the-skinny-...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,1016.0
41721,2002267_1,Baukasten-Hose,https://www.c-and-a.com/de/de/shop/hose-200226...,https://www.c-and-a.com/productimages/b_rgb:EB...,.jpg,1016.0


In [150]:
pd.merge(
    df_items, 
    cats[['Category', 'Parent_Category']], 
    left_on='Category', 
    right_index=True, 
    how='left').groupby('Parent_Category').agg({
        'Code':['count', 'nunique']
    })

Code        
                count nunique
Parent_Category              
2.0               705     387
3.0               320     314
4.0               772     672
5.0               691     683
6.0               353     329
...               ...     ...
880.0              49      49
881.0             122      82
882.0              56      54
883.0              19      19
884.0              21      21

[76 rows x 2 columns]

In [151]:
# Get any failed images
i = 0
for idx, row in df_items.iterrows():
    res = save_image(row['Code'], row['Img_URL'], row['Img_Ext'], verbose=True)
    if res > 0:
        i += 1
        if i%10 == 0:
            time.sleep(1)

Iterate through each filter for each sub-category - get all product ids and tag them

In [153]:
filter_item_code = lambda x: f"{x.get('data-usim')}_{x.get('data-color')}"

In [165]:
size = 2000

if os.path.exists(filename_format.format('FilterAttributes')):
    df_attributes = pd.read_parquet(filename_format.format('FilterAttributes'))
else:
    df_attributes = pd.DataFrame(columns=['Code', 'Filter_ID'])

i = 0
total = filters_t.shape[0]

print(f'Total Filters = {total}')

for idx, row in filters_t[filters_t['Filter_Name'] != 'Farbe'].iterrows():
    if df_attributes[df_attributes['Filter_ID'] == idx].shape[0] > 0:
        # already completed,
        i += 1
        if i%10 == 0:        
            print(f"Skipped {i}")
        continue
    
    cat_filter = get_url(f"{cats.loc[int(row['Parent_Category'])]['URL']}?pagesize={size}&facets={row['Filter_Option'][1]}")
    codes = list(map(filter_item_code, cat_filter.find_all('div', attrs={'class': 'product-tile'})))
    df_attributes = pd.concat([
        df_attributes,
        pd.DataFrame({
            'Code': codes,
            'Filter_ID': [idx]*len(codes)
        })
    ])
    
    i += 1
    if i%10 == 0:
        df_attributes.to_parquet(filename_format.format('FilterAttributes'))
        print(f"Completed {i}")
        time.sleep(1)

df_attributes.to_parquet(filename_format.format('FilterAttributes'))
print("All Done!")

Total Filters = 3203
Completed 10
Completed 20
Completed 30
Completed 40
Completed 50
Completed 60
Completed 70
Completed 80
Completed 90
Completed 100
Completed 110
Completed 120
Completed 130
Completed 140
Completed 150
Completed 160
Completed 170
Completed 180
Completed 190
Completed 200
Completed 210
Completed 220
Completed 230
Completed 240
Completed 250
Completed 260
Completed 270
Completed 280
Completed 290
Completed 300
Completed 310
Completed 320
Completed 330
Completed 340
Completed 350
Completed 360
Completed 370
Completed 380
Completed 390
Completed 400
Completed 410
Completed 420
Completed 430
Completed 440
Completed 450
Completed 460
Completed 470
Completed 480
Completed 490
Completed 500
Completed 510
Completed 520
Completed 530
Completed 540
Completed 550
Completed 560
Completed 570
Completed 580
Completed 590
Completed 600
Completed 610
Completed 620
Completed 630
Completed 640
Completed 650
Completed 660
Completed 670
Completed 680
Completed 690
Completed 700
Complete

In [166]:
df_attributes

,Code,Filter_ID
0,2149117_1,579
1,2131540_1,579
2,2129608_1,579
3,2101813_1,579
4,2132280_2,579
...,...,...
14,2012129_1,9023
15,2012156_1,9023
16,2010490_1,9023
17,2033199_2,9023


In [167]:
df_attributes['Code'].nunique()

8710

Removing some images which would probably not be good for training

In [ ]:
# bad_images = [
#     '0111565001', '0111593001', '0156227001', '0156231001', '0158340001', '0228257001', '0228257002',
#     '0228257003', '0228257004', '0228257017', '0228257018', '0542413008', '0542413012', '0561274001',
#     '0987624001', '0987624004', '0987624005', '0987628001', '0987628002', '0987628003', '0987628004', 
#     '0987628006'
# ]